# Commercial Register Exploration

In [1]:
import requests
import fake_useragent
from datetime import datetime
from bs4 import BeautifulSoup, Comment
from soup_helpers import get_view_state, get_session_id
from cr_retriever import CommercialRegisterRetriever

In [2]:
cr = CommercialRegisterRetriever()

print(cr.get_jsession_id())

7895EAAD189658E0492B5562B8DBDB83.web03-1


In [6]:
ua = fake_useragent.UserAgent()
ua_chrome = ua.chrome

base_url = "https://www.unternehmensregister.de/ureg/"

headers = {
  'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': ua_chrome,
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

payload = {}



## Get SessionID

In [7]:
response = requests.request("GET", base_url, headers=headers, data=payload)

print(response.status_code)

200


In [5]:
jsessionid = get_session_id(response.text)
view_state = get_view_state(BeautifulSoup(response.text, 'html.parser'))
print(view_state, jsessionid)

-4717578002224379934:7839400966504794595 B2A5E352EF10D2AE5118CCA6A08DC4E9.web03-1


## Search for Company

In [165]:
import requests

search_term = "Finn"
url = f"{base_url}/index.html;jsessionid={jsessionid}"

payload = f'globalSearchForm=globalSearchForm&globalSearchForm%3AextendedResearchCompanyName={search_term}&globalSearchForm%3Ahidden_element_used_for_validation_only=aaa&submitaction=globalsearch&globalsearch=true&globalSearchForm%3AbtnExecuteSearchOld=Suchen&javax.faces.ViewState={view_state}'
headers = {
  'Host': 'www.unternehmensregister.de',
  'Origin': 'https://www.unternehmensregister.de',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
  'User-Agent': ua_chrome,
  'Referer': 'https://www.unternehmensregister.de/ureg/',
  'Accept-Language': 'en-GB,en;q=0.9',
  'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [166]:
url = f"https://www.unternehmensregister.de/ureg/result.html;jsessionid={jsessionid}"

payload = {}
headers = {
  'Host': 'www.unternehmensregister.de',
  'Cache-Control': 'max-age=0',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Referer': f'https://www.unternehmensregister.de/ureg/index.html;jsessionid={jsessionid}',
  'Accept-Language': 'en-DE,en;q=0.9,de-DE;q=0.8,de;q=0.7,da-DK;q=0.6,da;q=0.5,en-US;q=0.4'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


In [167]:
# extract data
from bs4 import BeautifulSoup

html = response.text
soup = BeautifulSoup(html, "html.parser")

# Extract company name
company_name = soup.find("div", class_="company_result").find("b").get_text(strip=True)

# Assuming 'soup' is your BeautifulSoup object
company_link = soup.find("a", string="Registerinformationen des Registergerichts")["href"]

# Use regex to find company number
company_number = re.search(r'company=(\d+)', company_link)

# Check if a match was found
if company_number:
    company_id = company_number.group(1)  # Group 1 contains the first set of parentheses matches
else:
    company_id = None


print(company_name)
print(company_id)

Finner Hoch- und Tiefbauunternehmung GmbH
9811


## Load Register Information

In [168]:
import requests

url = f"https://www.unternehmensregister.de/ureg/registerPortalAdvice.html;jsessionid={jsessionid}?submitaction=regnav&company={company_id}"

payload = {}
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


## Loop Through Document Tree to find Gesellschafterliste


In [169]:
url = f"https://www.unternehmensregister.de/ureg/registerPortal.html;jsessionid={jsessionid}?submitaction=showDkTree&searchIdx=0"

payload = {}
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


In [170]:
soup = BeautifulSoup(response.text, 'html.parser')

for a in soup.find_all('a'):
    i = a.find('i', title="Öffnet diesen Baum des Dokumentenbaums")
    if i is not None:
        href_value = a['href']
        break
else:
    href_value = None

submit_action = href_value
print(submit_action)

?submitaction=showDkTree&gkz=B1601&gkzalt=&rtype=HRB&rnr=103526&action=tree&id=N__0_0&state=1&addedId=N__0_0&searchIdx=0


In [171]:
url = f"https://www.unternehmensregister.de/ureg/registerPortal.html;jsessionid={jsessionid}{submit_action}"

payload = {}
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


In [172]:
soup = BeautifulSoup(response.text, 'html.parser')

tag_a = soup.find('a', string="Liste der Gesellschafter")
href_value = tag_a['href'] if tag_a else None
submit_action = href_value
print(submit_action)

?submitaction=showDkTree&action=select&addedId=N__0_0_0&searchIdx=0


In [173]:
url = f"https://www.unternehmensregister.de/ureg/registerPortal.html;jsessionid={jsessionid}{submit_action}"

payload = {}
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)

200


In [174]:
soup = BeautifulSoup(response.text, 'html.parser')

date_format = "%d.%m.%Y"

# Initialize the latest_date and the href_value
latest_date = datetime.min
href_value = ""

# Find all the 'a' tags with "Liste der Gesellschafter"
tags_a = soup.find_all('a', string=lambda x: x and "Liste der Gesellschafter" in x)

# Loop through all the tags
for tag_a in tags_a:
    # find date in the format dd.mm.yyyy
    date_string = re.search(r'\d{2}\.\d{2}\.\d{4}', tag_a.text)
    if date_string:
        date = datetime.strptime(date_string.group(), date_format)
        if date > latest_date:
            latest_date = date
            href_value = tag_a['href']
            file_name = tag_a.text

print(latest_date)
print(href_value)

submit_action = href_value




2020-09-04 00:00:00
?submitaction=showDkTree&action=select&addedId=N__0_0_0_0&searchIdx=0


In [175]:
url = f"https://www.unternehmensregister.de/ureg/registerPortal.html;jsessionid={jsessionid}{submit_action}"

payload = {}
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)

200


## Add Document to Warenkorb

In [176]:
print(response.text)

<!DOCTYPE html>
<html lang="de" xmlns="http://www.w3.org/1999/xhtml"><head id="j_idt2">
    <title>Unternehmensregister</title>
    <meta charset="UTF-8" />
    <link rel="stylesheet" href="assets/css/style.min.css" />
    <!-- link rel="stylesheet" type="text/css" href="/ureg/styles/regport.css" media="screen"/ -->
    <link rel="stylesheet" type="text/css" href="/ureg/styles/main-redesign.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/cc.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/plus.css" media="screen" />

    <script type="text/javascript">
          var tooltip_str_print = "Drucken";
          var tooltip_str_close = "Schließen";
          var tooltip_str_image = "Bild";
          var tooltip_str_of    = "von";
          var tooltip_str_next  = "Nächstes";
          var tooltip_str_prev  = "Vorheriges";
          var tooltip_str_loading = "Bilddaten werden geladen …";
          var tooltip_str_error =

In [177]:
soup = BeautifulSoup(response.text, 'html.parser')
tag_td = soup.find('td', class_='submit-container')

inputs = {}

# Find all 'input' tags and store their 'id' and 'value' as key-value pairs in the dictionary
for input_tag in soup.find_all('input'):
    inputs[input_tag.get('id')] = input_tag.get('value')

# Now you can access each input value as a variable using its id. For example, to get the value of 'submitaction':
submitaction = inputs.get('submitaction')

# Print the variable
print(inputs)


{None: 'Suchen', 'loginForm:username': None, 'loginForm:password': '', 'loginForm:btnLogin': 'Anmelden', 'j_id1:javax.faces.ViewState:0': '-2936701593460514553:-5883974387246202109', 'searchRegisterForm:registerDataCompanyName': 'Finner Hoch- und Tiefbauunternehmung GmbH', 'searchRegisterForm:registerDataSearchEntrysCheck': None, 'searchRegisterForm:registerDataCompanyLocation': '', 'searchRegisterForm:registerDataRegisterNumber': '103526', 'searchRegisterForm:searchFederalState1': None, 'searchRegisterForm:searchFederalState2': None, 'searchRegisterForm:searchFederalState3': None, 'searchRegisterForm:searchFederalState4': None, 'searchRegisterForm:searchFederalState5': None, 'searchRegisterForm:searchFederalState6': None, 'searchRegisterForm:searchFederalState7': None, 'searchRegisterForm:searchFederalState8': None, 'searchRegisterForm:searchFederalState9': None, 'searchRegisterForm:searchFederalState10': None, 'searchRegisterForm:searchFederalState11': None, 'searchRegisterForm:searc

In [178]:
import urllib.parse

keys = ['ausdruckart', 'cname', 'docnames', 'format', 'gkz', 'gkzalt', 'hasChildren', 'iddok', 'location', 'rnr', 'rtype', 'searchIdx', 'state', 'submitaction']

payload = {key: value for key, value in inputs.items() if value is not None and key in keys}
payload = urllib.parse.urlencode(payload)
print(payload)

url = f"https://www.unternehmensregister.de/ureg/registerPortal.html;jsessionid={jsessionid}"

# payload = f'ausdruckart=DK_ID&cname=Tanso%20Technologies%20GmbH&docnames=N__0_0_1_0&format=pdf&gkz=D2601&gkzalt=&hasChildren=true&iddok=addedId&location=&rnr=269123&rtype=HRB&searchIdx=0&state=2&submitaction=ajax_check4duplicate'
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'Accept': '*/*',
  'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
  'X-Requested-With': 'XMLHttpRequest',
  'sec-ch-ua-mobile': '?0',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'sec-ch-ua-platform': '"macOS"',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Dest': 'empty',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.status_code)


submitaction=check4duplicate&iddok=addedId&ausdruckart=DK_ID&gkz=B1601&rtype=HRB&hasChildren=true&rnr=103526&cname=Finner+Hoch-+und+Tiefbauunternehmung+GmbH&state=1&docnames=N__0_0_0_0&searchIdx=0
200


In [179]:
import requests

url = f"https://www.unternehmensregister.de/ureg/doccart.html;jsessionid={jsessionid}"

payload = {}
headers = {
  'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


In [180]:
print(response.text)

soup = BeautifulSoup(response.text, 'html.parser')

print(check_if_basket_has_content(soup))

<!DOCTYPE html>
<html lang="de" xmlns="http://www.w3.org/1999/xhtml"><head id="j_idt2">
    <title>Unternehmensregister</title>
    <meta charset="UTF-8" />
    <link rel="stylesheet" href="assets/css/style.min.css" />
    <!-- link rel="stylesheet" type="text/css" href="/ureg/styles/regport.css" media="screen"/ -->
    <link rel="stylesheet" type="text/css" href="/ureg/styles/main-redesign.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/cc.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/plus.css" media="screen" />

    <script type="text/javascript">
          var tooltip_str_print = "Drucken";
          var tooltip_str_close = "Schließen";
          var tooltip_str_image = "Bild";
          var tooltip_str_of    = "von";
          var tooltip_str_next  = "Nächstes";
          var tooltip_str_prev  = "Vorheriges";
          var tooltip_str_loading = "Bilddaten werden geladen …";
          var tooltip_str_error =

In [181]:
# Parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find the input element with name="javax.faces.ViewState"
view_state_input = soup.find('input', {'name': 'javax.faces.ViewState'})

# Extract the value attribute
view_state_value = view_state_input['value']
input_element = soup.find('input', {'name': lambda value: value and value.startswith('transactionref_')})
transaction_ref = input_element['name']

print(view_state_value)
print(transaction_ref)



-8144195337107605713:7277943706064500240
transactionref_B1601_HRB_103526_DK_ID_1_1688140953463


In [182]:
payload = {'doccartForm': 'doccartForm', 'doccartForm:j_idt351': 'Zur Freischaltung', 'javax.faces.ViewState': view_state_value, transaction_ref: ''}
payload = urllib.parse.urlencode(payload)

url = f"https://www.unternehmensregister.de/ureg/doccart.html;jsessionid={jsessionid}"



# payload = f'doccartForm=doccartForm&doccartForm%3Aj_idt351=Zur%20Freischaltung&javax.faces.ViewState=-6634781701497412073%3A-5381120811431848088&transactionref_F1103_HRB_229700_DK_ID_1_1688135675298='
headers = {
  'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'Content-Type': 'application/x-www-form-urlencoded',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


<!DOCTYPE html>
<html lang="de" xmlns="http://www.w3.org/1999/xhtml"><head id="j_idt2">
    <title>Unternehmensregister</title>
    <meta charset="UTF-8" />
    <link rel="stylesheet" href="assets/css/style.min.css" />
    <!-- link rel="stylesheet" type="text/css" href="/ureg/styles/regport.css" media="screen"/ -->
    <link rel="stylesheet" type="text/css" href="/ureg/styles/main-redesign.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/cc.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/plus.css" media="screen" />

    <script type="text/javascript">
          var tooltip_str_print = "Drucken";
          var tooltip_str_close = "Schließen";
          var tooltip_str_image = "Bild";
          var tooltip_str_of    = "von";
          var tooltip_str_next  = "Nächstes";
          var tooltip_str_prev  = "Vorheriges";
          var tooltip_str_loading = "Bilddaten werden geladen …";
          var tooltip_str_error =

In [183]:
import requests

url = f"https://www.unternehmensregister.de/ureg/payment_overview.html;jsessionid={jsessionid}"

payload = {}
headers = {
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


In [184]:
soup = BeautifulSoup(response.text, 'html.parser')
print(check_if_basket_has_content(soup))

True


In [185]:
soup = BeautifulSoup(response.text, 'html.parser')
view_state_input = soup.find('input', {'name': 'javax.faces.ViewState'})
view_state_value = view_state_input['value']
print(view_state_value)

-4470892931352494050:8425942236257059065


In [186]:
import requests

url = f"https://www.unternehmensregister.de/ureg/payment_overview.html;jsessionid={jsessionid}"

payload = {
    'javax.faces.ViewState': view_state_value,
    'paymentFormOverview': 'paymentFormOverview',
    'paymentFormOverview:acceptAGB': 'true',
    'paymentFormOverview:btnNext': 'Jetzt freischalten',
    'submitAmount': '0',
    'submitaction': 'paymentSubmit'
}
payload = urllib.parse.urlencode(payload)

# payload = f'javax.faces.ViewState=-7031313347653954904%3A2884316281357227295&paymentFormOverview=paymentFormOverview&paymentFormOverview%3AacceptAGB=true&paymentFormOverview%3AbtnNext=Jetzt%20freischalten&submitAmount=0&submitaction=paymentSubmit'
headers = {
  'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'Content-Type': 'application/x-www-form-urlencoded',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.status_code)


200


In [187]:
soup = BeautifulSoup(response.text, 'html.parser')
print(check_if_basket_has_content(soup))

False


In [188]:
import requests

url = f"https://www.unternehmensregister.de/ureg/payment_confirm2.html;jsessionid={jsessionid}"

payload = {}
headers = {
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


In [189]:
soup = BeautifulSoup(response.text, 'html.parser')
print(check_if_basket_has_content(soup))

False


In [190]:
import requests

url = f"https://www.unternehmensregister.de/ureg/doccart.html;jsessionid={jsessionid}"

payload = {}
headers = {
  'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


In [191]:
soup = BeautifulSoup(response.text, 'html.parser')

# Find the <div> element with id "order-collapse1"
div_element = soup.find('div', id='order-collapse1')

# Find the nested <div> element with class "document"
nested_div = div_element.find('div', class_='document')

# Extract the text within the nested <div> element
text = nested_div.get_text(strip=True)

print(text)

MengeDokumente


In [192]:
print(response.text)

<!DOCTYPE html>
<html lang="de" xmlns="http://www.w3.org/1999/xhtml"><head id="j_idt2">
    <title>Unternehmensregister</title>
    <meta charset="UTF-8" />
    <link rel="stylesheet" href="assets/css/style.min.css" />
    <!-- link rel="stylesheet" type="text/css" href="/ureg/styles/regport.css" media="screen"/ -->
    <link rel="stylesheet" type="text/css" href="/ureg/styles/main-redesign.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/cc.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/plus.css" media="screen" />

    <script type="text/javascript">
          var tooltip_str_print = "Drucken";
          var tooltip_str_close = "Schließen";
          var tooltip_str_image = "Bild";
          var tooltip_str_of    = "von";
          var tooltip_str_next  = "Nächstes";
          var tooltip_str_prev  = "Vorheriges";
          var tooltip_str_loading = "Bilddaten werden geladen …";
          var tooltip_str_error =

In [193]:
soup = BeautifulSoup(response.text, 'html.parser')

# Find the <div> element with the class "download-wrapper"
div_element = soup.find('div', class_='download-wrapper')

# Find the <a> element within the <div> element
a_element = div_element.find('a')

# Extract the value of the href attribute
href_value = a_element['href']
if 'format=null' in href_value:
    href_value = href_value.replace('format=null', 'format=pdf')
else:
    href_value += '&format=pdf'




print(href_value)

registerdocument.zip;jsessionid=05C02954147A99CFB7D7F95AE991F804.web02-1?iddok=27935038&submitaction=registerdocument&ausdruckart=DK_ID&gkz=B1601&gkzalt=&rtype=HRB&rnr=103526&state=1&searchIdx=0&docId=B1601_HRB_103526_DK_ID_1_1688140953463&format=pdf


## Download Document

In [194]:
import requests

# url = f"https://www.unternehmensregister.de/ureg/registerdocument.zip;jsessionid={jsessionid}?iddok=17634100&submitaction=registerdocument&ausdruckart=DK_ID&gkz=F1103&gkzalt=&rtype=HRB&rnr=229700&state=3&searchIdx=0&docId=F1103_HRB_229700_DK_ID_1_1688135675298&format=pdf"
url = f"https://www.unternehmensregister.de/ureg/{href_value}"


payload = {}
headers = {
  'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


In [195]:
keys = ['ausdruckart', 'cname', 'docnames', 'format', 'gkz', 'gkzalt', 'hasChildren', 'iddok', 'location', 'rnr', 'rtype', 'searchIdx', 'state', 'submitaction']
current_datetime = datetime.now()


file_store_name = inputs['rnr'] + "_" + inputs['cname'] + "_" + file_name + '_' + current_datetime.isoformat() + '.pdf'

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Save the response content to a file
    with open(file_store_name, 'wb') as file:
        file.write(response.content)
    print('File successfully retrieved and saved.')
else:
    print('Error occurred while retrieving the file.')

File successfully retrieved and saved.


'2023-06-30T17:56:38.807453'